In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import numpy as np

ar_df = pd.read_csv("air_reserve.csv")#餐廳預約日期&造訪日期&預約人數
asi_df = pd.read_csv("air_store_info.csv")#餐廳種類&地址&經緯度
avd_df = pd.read_csv("air_visit_data.csv")#造訪日期&人數
di_df = pd.read_csv("date_info.csv")#是否假日
hr_df = pd.read_csv("hpg_reserve.csv")
hsi_df = pd.read_csv("hpg_store_info.csv")
sir_df = pd.read_csv("store_id_relation.csv")

In [2]:
ar_df["reserve_datetime"] = pd.to_datetime(ar_df["reserve_datetime"], format='%Y-%m-%d %H:%M:%S', errors='coerce')
ar_df["reserve_date"] = ar_df["reserve_datetime"].dt.date

In [3]:
ar_df["visit_datetime"] = pd.to_datetime(ar_df["visit_datetime"], format='%Y-%m-%d %H:%M:%S', errors='coerce')
# ar_df["visit_years"] = ar_df["visit_datetime"].dt.year
ar_df["visit_date"] = ar_df["visit_datetime"].dt.date
# ar_df["dow"] = ar_df["visit_datetime"].dt.dayofweek
ar_df = ar_df.drop(['visit_datetime', 'reserve_datetime'], axis=1)
ar_df

,air_store_id,reserve_visitors,reserve_date,visit_date
0,air_877f79706adbfb06,1,2016-01-01,2016-01-01
1,air_db4b38ebe7a7ceff,3,2016-01-01,2016-01-01
2,air_db4b38ebe7a7ceff,6,2016-01-01,2016-01-01
3,air_877f79706adbfb06,2,2016-01-01,2016-01-01
4,air_db80363d35f10926,5,2016-01-01,2016-01-01
...,...,...,...,...
92373,air_db4b38ebe7a7ceff,6,2017-04-17,2017-05-29
92374,air_1033310359ceeac1,8,2017-04-19,2017-05-30
92375,air_877f79706adbfb06,3,2017-04-12,2017-05-31
92376,air_900d755ebd2f7bbd,10,2017-04-19,2017-05-31


In [4]:
ar_df["reserve_diff"] = ar_df.apply(lambda x: (x['visit_date'] - x['reserve_date']).days, axis=1)
tmp2 = ar_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_diff'].mean().rename(columns = {'reserve_diff':'mean_reserve_diff'})
tmp2

,air_store_id,visit_date,mean_reserve_diff
0,air_00a91d42b08b08d9,2016-10-31,0.0
1,air_00a91d42b08b08d9,2016-12-05,4.0
2,air_00a91d42b08b08d9,2016-12-14,6.0
3,air_00a91d42b08b08d9,2016-12-17,6.0
4,air_00a91d42b08b08d9,2016-12-20,2.0
...,...,...,...
29825,air_fea5dc9594450608,2017-04-22,0.0
29826,air_fea5dc9594450608,2017-04-25,9.0
29827,air_fea5dc9594450608,2017-04-28,29.0
29828,air_fea5dc9594450608,2017-05-20,34.0


In [5]:
# 依照visit_date合併有預約的人數
tmp = ar_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_visitors'].sum().rename(columns = {'reserve_visitors':'sum_reserve_visitors'})
tmp = tmp[['air_store_id', 'visit_date', 'sum_reserve_visitors']]
tmp1 = ar_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_visitors'].mean().rename(columns = {'reserve_visitors':'mean_reserve_visitors'})
train = pd.merge(tmp, tmp1, on = ['air_store_id', 'visit_date'], how = 'inner')
train = pd.merge(train, tmp2, on = ['air_store_id', 'visit_date'], how = 'inner')

In [6]:
train

,air_store_id,visit_date,sum_reserve_visitors,mean_reserve_visitors,mean_reserve_diff
0,air_00a91d42b08b08d9,2016-10-31,2,2.0,0.0
1,air_00a91d42b08b08d9,2016-12-05,9,9.0,4.0
2,air_00a91d42b08b08d9,2016-12-14,18,18.0,6.0
3,air_00a91d42b08b08d9,2016-12-17,2,2.0,6.0
4,air_00a91d42b08b08d9,2016-12-20,4,4.0,2.0
...,...,...,...,...,...
29825,air_fea5dc9594450608,2017-04-22,2,2.0,0.0
29826,air_fea5dc9594450608,2017-04-25,2,2.0,9.0
29827,air_fea5dc9594450608,2017-04-28,3,3.0,29.0
29828,air_fea5dc9594450608,2017-05-20,6,6.0,34.0


In [7]:
#加入實際造訪人數
avd_df['visit_date'] = pd.to_datetime(avd_df['visit_date'], format = '%Y-%m-%d', errors= 'coerce')
avd_df['dow'] = avd_df['visit_date'].dt.dayofweek
tmp2 = avd_df.groupby(['air_store_id', 'dow'], as_index = False)['visitors'].mean().rename(columns = {'visitors' : 'mean_visitors'})
avd_df = pd.merge(avd_df, tmp2, on = ['air_store_id', 'dow'])
tmp3 = avd_df.groupby(['air_store_id', 'dow'], as_index = False)['visitors'].max().rename(columns = {'visitors' : 'max_visitors'})
avd_df = pd.merge(avd_df, tmp3, on = ['air_store_id', 'dow'])
tmp4 = avd_df.groupby(['air_store_id', 'dow'], as_index = False)['visitors'].min().rename(columns = {'visitors' : 'min_visitors'})
avd_df = pd.merge(avd_df, tmp4, on = ['air_store_id', 'dow'])
tmp5 = avd_df.groupby(['air_store_id', 'dow'], as_index = False)['visitors'].median().rename(columns = {'visitors' : 'median_visitors'})
avd_df = pd.merge(avd_df, tmp5, on = ['air_store_id', 'dow'])
tmp6 = avd_df.groupby(['air_store_id', 'dow'], as_index = False)['visitors'].count().rename(columns = {'visitors' : 'count_visitors'})
avd_df = pd.merge(avd_df, tmp6, on = ['air_store_id', 'dow'])
avd_df

,air_store_id,visit_date,visitors,dow,mean_visitors,max_visitors,min_visitors,median_visitors,count_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,2,23.843750,57,7,25.0,64
1,air_ba937bf13d40fb24,2016-01-20,31,2,23.843750,57,7,25.0,64
2,air_ba937bf13d40fb24,2016-01-27,24,2,23.843750,57,7,25.0,64
3,air_ba937bf13d40fb24,2016-02-03,18,2,23.843750,57,7,25.0,64
4,air_ba937bf13d40fb24,2016-02-10,32,2,23.843750,57,7,25.0,64
...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-13,6,3,5.394737,8,2,6.0,38
252104,air_24e8414b9b07decb,2017-04-20,7,3,5.394737,8,2,6.0,38
252105,air_24e8414b9b07decb,2016-07-17,1,6,3.666667,6,1,4.0,3
252106,air_24e8414b9b07decb,2016-12-25,6,6,3.666667,6,1,4.0,3


In [8]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
train = pd.merge(avd_df, train, how = 'left', on = ['air_store_id', 'visit_date']) 
train = train.fillna(-1)

In [9]:
hr_df = pd.merge(hr_df, sir_df, how='inner', on=['hpg_store_id'])

In [10]:
# 將hpg_reserve中visit_datetime&reserve_datetime時間去掉
hr_df["reserve_datetime"] = pd.to_datetime(hr_df["reserve_datetime"], format='%Y-%m-%d %H:%M:%S', errors='coerce')
hr_df["reserve_date"] = hr_df["reserve_datetime"].dt.date

In [11]:
hr_df["visit_datetime"] = pd.to_datetime(hr_df["visit_datetime"], format='%Y-%m-%d %H:%M:%S', errors='coerce')
hr_df["visit_years"] = hr_df["visit_datetime"].dt.year
hr_df["visit_date"] = hr_df["visit_datetime"].dt.date

In [12]:
hr_df["reserve_diff"] = hr_df.apply(lambda x: (x['visit_date'] - x['reserve_date']).days, axis=1)

In [13]:
hr_df

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id,reserve_date,visit_years,visit_date,reserve_diff
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926,2016-01-01,2016,2016-01-01,0
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926,2016-01-02,2016,2016-01-02,0
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926,2016-01-02,2016,2016-01-03,1
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926,2016-01-04,2016,2016-01-06,2
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926,2016-01-11,2016,2016-01-11,0
...,...,...,...,...,...,...,...,...,...
28178,hpg_0f78ef8d1e4b23d8,2017-04-01 21:00:00,2017-03-10 20:00:00,2,air_84f6876ff7e83ae7,2017-03-10,2017,2017-04-01,22
28179,hpg_0f78ef8d1e4b23d8,2017-04-02 17:00:00,2017-04-02 14:00:00,2,air_84f6876ff7e83ae7,2017-04-02,2017,2017-04-02,0
28180,hpg_0f78ef8d1e4b23d8,2017-04-08 19:00:00,2017-04-03 12:00:00,3,air_84f6876ff7e83ae7,2017-04-03,2017,2017-04-08,5
28181,hpg_0f78ef8d1e4b23d8,2017-04-08 20:00:00,2017-03-09 07:00:00,4,air_84f6876ff7e83ae7,2017-03-09,2017,2017-04-08,30


In [14]:
hr_df = hr_df.drop(['visit_datetime', 'reserve_datetime', 'hpg_store_id'], axis=1)

In [15]:
hr_df

,reserve_visitors,air_store_id,reserve_date,visit_years,visit_date,reserve_diff
0,4,air_db80363d35f10926,2016-01-01,2016,2016-01-01,0
1,2,air_db80363d35f10926,2016-01-02,2016,2016-01-02,0
2,6,air_db80363d35f10926,2016-01-02,2016,2016-01-03,1
3,3,air_db80363d35f10926,2016-01-04,2016,2016-01-06,2
4,2,air_db80363d35f10926,2016-01-11,2016,2016-01-11,0
...,...,...,...,...,...,...
28178,2,air_84f6876ff7e83ae7,2017-03-10,2017,2017-04-01,22
28179,2,air_84f6876ff7e83ae7,2017-04-02,2017,2017-04-02,0
28180,3,air_84f6876ff7e83ae7,2017-04-03,2017,2017-04-08,5
28181,4,air_84f6876ff7e83ae7,2017-03-09,2017,2017-04-08,30


In [16]:
# 依照visit_date合併有預約的人數
tmp = hr_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_visitors'].sum().rename(columns = {'reserve_visitors':'sum_reserve_visitors'})
tmp

,air_store_id,visit_date,sum_reserve_visitors
0,air_00a91d42b08b08d9,2016-01-14,2
1,air_00a91d42b08b08d9,2016-01-15,4
2,air_00a91d42b08b08d9,2016-01-16,2
3,air_00a91d42b08b08d9,2016-01-22,2
4,air_00a91d42b08b08d9,2016-01-29,5
...,...,...,...
18615,air_fea5dc9594450608,2017-04-21,2
18616,air_fea5dc9594450608,2017-04-22,2
18617,air_fea5dc9594450608,2017-04-25,2
18618,air_fea5dc9594450608,2017-04-28,3


In [17]:
# 依照visit_date合併人數
tmp1 = hr_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_visitors'].mean().rename(columns = {'reserve_visitors':'mean_reserve_visitors'})
train_tmp = pd.merge(tmp, tmp1, on = ['air_store_id', 'visit_date'], how = 'inner')
tmp2 = hr_df.groupby(['air_store_id', 'visit_date'], as_index = False)['reserve_diff'].mean().rename(columns = {'reserve_diff':'mean_reserve_diff'})
train_tmp = pd.merge(train_tmp, tmp2, on = ['air_store_id', 'visit_date'], how = 'inner')
train_tmp

,air_store_id,visit_date,sum_reserve_visitors,mean_reserve_visitors,mean_reserve_diff
0,air_00a91d42b08b08d9,2016-01-14,2,2.0,3.0
1,air_00a91d42b08b08d9,2016-01-15,4,4.0,6.0
2,air_00a91d42b08b08d9,2016-01-16,2,2.0,3.0
3,air_00a91d42b08b08d9,2016-01-22,2,2.0,3.0
4,air_00a91d42b08b08d9,2016-01-29,5,5.0,6.0
...,...,...,...,...,...
18615,air_fea5dc9594450608,2017-04-21,2,2.0,2.0
18616,air_fea5dc9594450608,2017-04-22,2,2.0,0.0
18617,air_fea5dc9594450608,2017-04-25,2,2.0,9.0
18618,air_fea5dc9594450608,2017-04-28,3,3.0,29.0


In [18]:
train_tmp['visit_date'] = pd.to_datetime(train_tmp['visit_date'])
train_tmp1 = pd.merge(avd_df, train_tmp, how = 'left', on = ['air_store_id', 'visit_date']) 
train_tmp1

,air_store_id,visit_date,visitors,dow,mean_visitors,max_visitors,min_visitors,median_visitors,count_visitors,sum_reserve_visitors,mean_reserve_visitors,mean_reserve_diff
0,air_ba937bf13d40fb24,2016-01-13,25,2,23.843750,57,7,25.0,64,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-20,31,2,23.843750,57,7,25.0,64,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-27,24,2,23.843750,57,7,25.0,64,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-02-03,18,2,23.843750,57,7,25.0,64,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-02-10,32,2,23.843750,57,7,25.0,64,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-13,6,3,5.394737,8,2,6.0,38,NaN,NaN,NaN
252104,air_24e8414b9b07decb,2017-04-20,7,3,5.394737,8,2,6.0,38,NaN,NaN,NaN
252105,air_24e8414b9b07decb,2016-07-17,1,6,3.666667,6,1,4.0,3,NaN,NaN,NaN
252106,air_24e8414b9b07decb,2016-12-25,6,6,3.666667,6,1,4.0,3,NaN,NaN,NaN


In [19]:
train_tmp2 = pd.merge(train_tmp1, train_tmp, how = 'left', on = ['air_store_id', 'visit_date']) 
train_tmp2

,air_store_id,visit_date,visitors,dow,mean_visitors,max_visitors,min_visitors,median_visitors,count_visitors,sum_reserve_visitors_x,mean_reserve_visitors_x,mean_reserve_diff_x,sum_reserve_visitors_y,mean_reserve_visitors_y,mean_reserve_diff_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,23.843750,57,7,25.0,64,NaN,NaN,NaN,NaN,NaN,NaN
1,air_ba937bf13d40fb24,2016-01-20,31,2,23.843750,57,7,25.0,64,NaN,NaN,NaN,NaN,NaN,NaN
2,air_ba937bf13d40fb24,2016-01-27,24,2,23.843750,57,7,25.0,64,NaN,NaN,NaN,NaN,NaN,NaN
3,air_ba937bf13d40fb24,2016-02-03,18,2,23.843750,57,7,25.0,64,NaN,NaN,NaN,NaN,NaN,NaN
4,air_ba937bf13d40fb24,2016-02-10,32,2,23.843750,57,7,25.0,64,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-13,6,3,5.394737,8,2,6.0,38,NaN,NaN,NaN,NaN,NaN,NaN
252104,air_24e8414b9b07decb,2017-04-20,7,3,5.394737,8,2,6.0,38,NaN,NaN,NaN,NaN,NaN,NaN
252105,air_24e8414b9b07decb,2016-07-17,1,6,3.666667,6,1,4.0,3,NaN,NaN,NaN,NaN,NaN,NaN
252106,air_24e8414b9b07decb,2016-12-25,6,6,3.666667,6,1,4.0,3,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
train = train_tmp2.fillna(-1)

In [21]:
train

,air_store_id,visit_date,visitors,dow,mean_visitors,max_visitors,min_visitors,median_visitors,count_visitors,sum_reserve_visitors_x,mean_reserve_visitors_x,mean_reserve_diff_x,sum_reserve_visitors_y,mean_reserve_visitors_y,mean_reserve_diff_y
0,air_ba937bf13d40fb24,2016-01-13,25,2,23.843750,57,7,25.0,64,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-20,31,2,23.843750,57,7,25.0,64,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-27,24,2,23.843750,57,7,25.0,64,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-02-03,18,2,23.843750,57,7,25.0,64,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-02-10,32,2,23.843750,57,7,25.0,64,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252103,air_24e8414b9b07decb,2017-04-13,6,3,5.394737,8,2,6.0,38,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
252104,air_24e8414b9b07decb,2017-04-20,7,3,5.394737,8,2,6.0,38,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
252105,air_24e8414b9b07decb,2016-07-17,1,6,3.666667,6,1,4.0,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
252106,air_24e8414b9b07decb,2016-12-25,6,6,3.666667,6,1,4.0,3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
def split_fun1(x):
    lis = str(x).split('/')
    if len(lis) > 1:
        return lis[1]
    else:
        return lis[0]
 

In [23]:
def split_fun2(x):
    lis = str(x).split('/')
    if len(lis) > 2:
        return lis[2]
    else:
        return lis[0]

In [24]:
#新增餐廳種類&地址&經緯度
train_tmp = pd.merge(train, asi_df, on = 'air_store_id', how = 'left')
# 處理餐廳種類
train_tmp["genre_name0"] = train_tmp['air_genre_name'].apply(lambda x: str(x).split('/')[0])
train_tmp["genre_name1"] = train_tmp['air_genre_name'].apply(split_fun1)
train_tmp["genre_name2"] = train_tmp['air_genre_name'].apply(split_fun2)


In [25]:
# 處理餐廳地址
train_tmp["area_name0"] = train_tmp['air_area_name'].apply(lambda x: str(x).split(' ')[0])
train_tmp["area_name1"] = train_tmp['air_area_name'].apply(lambda x: str(x).split(' ')[1])
train_tmp["area_name2"] = train_tmp['air_area_name'].apply(lambda x: str(x).split(' ')[2])
train = train_tmp.copy()

In [26]:
train.columns

Index(['air_store_id', 'visit_date', 'visitors', 'dow', 'mean_visitors',
       'max_visitors', 'min_visitors', 'median_visitors', 'count_visitors',
       'sum_reserve_visitors_x', 'mean_reserve_visitors_x',
       'mean_reserve_diff_x', 'sum_reserve_visitors_y',
       'mean_reserve_visitors_y', 'mean_reserve_diff_y', 'air_genre_name',
       'air_area_name', 'latitude', 'longitude', 'genre_name0', 'genre_name1',
       'genre_name2', 'area_name0', 'area_name1', 'area_name2'],
      dtype='object')

In [27]:
lbl = preprocessing.LabelEncoder()

train['air_genre_name'] = lbl.fit_transform(train['air_genre_name'])
train['air_area_name'] = lbl.fit_transform(train['air_area_name'])
train['genre_name0'] = lbl.fit_transform(train['genre_name0'])
train['genre_name1'] = lbl.fit_transform(train['genre_name1'])
train['genre_name2'] = lbl.fit_transform(train['genre_name2'])
train['area_name0'] = lbl.fit_transform(train['area_name0'])
train['area_name1'] = lbl.fit_transform(train['area_name1'])
train['area_name2'] = lbl.fit_transform(train['area_name2'])

In [28]:
train['year'] = train['visit_date'].dt.year

In [29]:
lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
# test_b['air_store_id2'] = lbl.transform(test_b['air_store_id'])

In [30]:
train_a = train[train.loc[:,'year']==2016]
test_b = train[train.loc[:,'year']==2017]

In [31]:
train_a['date_int'] = train_a['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test_b['date_int'] = test_b['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)

/home/yoyo/miniconda3/envs/yooo/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/yoyo/miniconda3/envs/yooo/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
col = [c for c in train_a if c not in ['dow', 'year', 'id', 'air_store_id', 'reserve_date', 'visit_date', 'visitors', 'visit_datetime', 'reserve_datetime']]


In [33]:
col

['mean_visitors',
 'max_visitors',
 'min_visitors',
 'median_visitors',
 'count_visitors',
 'sum_reserve_visitors_x',
 'mean_reserve_visitors_x',
 'mean_reserve_diff_x',
 'sum_reserve_visitors_y',
 'mean_reserve_visitors_y',
 'mean_reserve_diff_y',
 'air_genre_name',
 'air_area_name',
 'latitude',
 'longitude',
 'genre_name0',
 'genre_name1',
 'genre_name2',
 'area_name0',
 'area_name1',
 'area_name2',
 'air_store_id2',
 'date_int']

In [34]:
train_a = train_a.fillna(-1)
test_b = test_b.fillna(-1)

In [35]:
train_a['visitors'] = np.log1p(train_a['visitors'].values)
test_b['visitors'] = np.log1p(test_b['visitors'].values)

In [36]:
X = train_a.copy()
X_test = test_b.copy()

In [37]:
# value_col = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'count_visitors'
#              ,'sum_reserve_visitors_x','mean_reserve_visitors_x', 'mean_reserve_diff_x'
#             ,'sum_reserve_visitors_y','mean_reserve_visitors_y', 'mean_reserve_diff_y']

In [38]:
value_scaler = preprocessing.MinMaxScaler()
for vcol in col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

In [39]:
X_train_tmp = X[col].to_numpy()
Y_train = X['visitors'].values
nn_test_tmp = X_test[col].to_numpy()
Y_test = X_test['visitors'].values

In [40]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
import tensorflow as tf

a = Input(shape=(X_train_tmp.shape[1],))
b1 = Dense(5, activation='relu', name='fc1')(a)
b2 = Dropout(0.5)(b1)
b3 = Dense(3, activation='relu', name='fc2')(b2)
b4 = Dropout(0.5)(b3)
b5 = Dense(3, activation='relu', name='fc3')(b4)

b6 = Dense(1)(b5)
model_1 = Model(inputs=a, outputs = b6)

Using TensorFlow backend.


In [41]:
a1 = Input(shape=(X_train_tmp.shape[1],))
b1 = Dense(5, activation='relu', name='fc_1')(a1)
b2 = Dense(3, activation='relu', name='fc_2')(b1)
b3 = Dense(1)(b2)
model_2 = Model(inputs=a1, outputs = b3)

In [42]:
optimizer1 = tf.keras.optimizers.Adam(0.001)
optimizer2 = tf.keras.optimizers.RMSprop(0.001)

model_1.compile(loss='mse',
                optimizer=optimizer1,
                metrics = ['mse'])
model_2.compile(loss='mse',
                optimizer=optimizer1,
                metrics = ['mse'])

In [43]:
history_1 = model_1.fit(X_train_tmp, Y_train, batch_size = 32, epochs=10, validation_split = 0.33)

Train on 116938 samples, validate on 57597 samples
Epoch 1/10
116938/116938 [==============================] - 8s 67us/step - loss: 1.5315 - mse: 1.5315 - val_loss: 0.5695 - val_mse: 0.5695
Epoch 2/10
116938/116938 [==============================] - 8s 71us/step - loss: 0.5760 - mse: 0.5760 - val_loss: 0.5315 - val_mse: 0.5315
Epoch 3/10
116938/116938 [==============================] - 8s 68us/step - loss: 0.5690 - mse: 0.5690 - val_loss: 0.5353 - val_mse: 0.5353
Epoch 4/10
116938/116938 [==============================] - 9s 75us/step - loss: 0.5681 - mse: 0.5681 - val_loss: 0.5285 - val_mse: 0.5285
Epoch 5/10
116938/116938 [==============================] - 9s 79us/step - loss: 0.5687 - mse: 0.5687 - val_loss: 0.5169 - val_mse: 0.5169
Epoch 6/10
116938/116938 [==============================] - 9s 77us/step - loss: 0.5690 - mse: 0.5690 - val_loss: 0.5428 - val_mse: 0.5428
Epoch 7/10
116938/116938 [==============================] - 10s 85us/step - loss: 0.5683 - mse: 0.5683 - val_loss: 

In [44]:
history_2 = model_2.fit(X_train_tmp, Y_train, batch_size = 32, epochs=10)

Epoch 1/10
174535/174535 [==============================] - 7s 40us/step - loss: 1.3461 - mse: 1.3461
Epoch 2/10
174535/174535 [==============================] - 8s 43us/step - loss: 0.6456 - mse: 0.6456
Epoch 3/10
174535/174535 [==============================] - 8s 45us/step - loss: 0.6456 - mse: 0.6456
Epoch 4/10
174535/174535 [==============================] - 8s 43us/step - loss: 0.6456 - mse: 0.6456
Epoch 5/10
174535/174535 [==============================] - 8s 44us/step - loss: 0.6456 - mse: 0.6456
Epoch 6/10
174535/174535 [==============================] - 8s 43us/step - loss: 0.6456 - mse: 0.6456
Epoch 7/10
174535/174535 [==============================] - 7s 42us/step - loss: 0.6456 - mse: 0.6456
Epoch 8/10
174535/174535 [==============================] - 8s 44us/step - loss: 0.6456 - mse: 0.6456
Epoch 9/10
174535/174535 [==============================] - 9s 50us/step - loss: 0.6456 - mse: 0.6456
Epoch 10/10
174535/174535 [==============================] - 8s 48us/step - loss: 

In [45]:
import sklearn.metrics as metrics
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

def actual_predicted(actual, predicted):
    print('RMSE: ', RMSLE(actual, predicted))


preds = pd.Series(model_1.predict(X_train_tmp).reshape(-1)).clip(0, 6.8).values
actual_output = train_a['visitors'].values
print("train:")
actual_predicted(actual_output, preds)

preds = pd.Series(model_1.predict(nn_test_tmp).reshape(-1)).values
actual_output = test_b['visitors'].values
print("test:")
actual_predicted(actual_output, preds)

preds = np.expm1(pd.Series(model_1.predict(X_train_tmp).reshape(-1)).values)
actual_output = np.expm1(train_a['visitors'].values)
print("train:")
actual_predicted(actual_output, preds)

preds = np.expm1(pd.Series(model_1.predict(nn_test_tmp).reshape(-1)).values)
actual_output = np.expm1(test_b['visitors'].values)
print("test:")
actual_predicted(actual_output, preds)

train:
RMSE:  0.7645386372775481
test:
RMSE:  0.7257524629914263
train:
RMSE:  16.42231705158677
test:
RMSE:  16.508609649315


In [46]:
preds = pd.Series(model_2.predict(X_train_tmp).reshape(-1)).clip(0, 6.8).values
actual_output = X['visitors'].values
print("train:")
actual_predicted(actual_output, preds)
preds = pd.Series(model_2.predict(nn_test_tmp).reshape(-1)).clip(0, 6.8).values
actual_output = X_test['visitors'].values
print("test:")
actual_predicted(actual_output, preds)

preds = np.expm1(pd.Series(model_2.predict(X_train_tmp).reshape(-1)).values)
actual_output = np.expm1(train_a['visitors'].values)
print("train:")
actual_predicted(actual_output, preds)

preds = np.expm1(pd.Series(model_2.predict(nn_test_tmp).reshape(-1)).values)
actual_output = np.expm1(test_b['visitors'].values)
print("test:")
actual_predicted(actual_output, preds)

train:
RMSE:  0.8036055141770763
test:
RMSE:  0.8171584591402069
train:
RMSE:  17.57142202923188
test:
RMSE:  17.949354029353294
